In [ ]:
!pip install torch torchvision scikit-learn tqdm matplotlib opencv-python

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tqdm import tqdm
import numpy as np


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [8]:
DATA_DIR = "dataset_split/eyes"

In [9]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [10]:
train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", transform=train_transforms)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/val", transform=val_test_transforms)
test_ds  = datasets.ImageFolder(f"{DATA_DIR}/test", transform=val_test_transforms)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

print("Classes:", train_ds.classes)


Classes: ['closed', 'open']


In [11]:
model = models.efficientnet_b0(weights="IMAGENET1K_V1")


In [12]:
model.classifier[1] = nn.Linear(
    model.classifier[1].in_features, 1
)

model = model.to(device)


In [ ]:
for param in model.features.parameters():
    param.requires_grad = False

In [14]:
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(
    model.classifier.parameters(),
    lr=1e-4
)


In [15]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, labels in tqdm(loader):
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)


In [16]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred, y_prob = [], [], []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)

            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > 0.5).astype(int)

            y_prob.extend(probs)
            y_pred.extend(preds)
            y_true.extend(labels.numpy())

    return np.array(y_true), np.array(y_pred), np.array(y_prob)


In [17]:
EPOCHS = 10

for epoch in range(EPOCHS):
    loss = train_one_epoch(model, train_loader)
    y_true, y_pred, _ = evaluate(model, val_loader)

    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("Train Loss:", round(loss, 4))
    print(classification_report(y_true, y_pred, target_names=train_ds.classes))


100%|██████████| 32/32 [00:38<00:00,  1.20s/it]



Epoch 1/10
Train Loss: 0.6546
              precision    recall  f1-score   support

      closed       0.81      0.46      0.59       108
        open       0.62      0.89      0.73       108

    accuracy                           0.68       216
   macro avg       0.71      0.68      0.66       216
weighted avg       0.71      0.68      0.66       216



100%|██████████| 32/32 [00:26<00:00,  1.19it/s]



Epoch 2/10
Train Loss: 0.6144
              precision    recall  f1-score   support

      closed       0.85      0.69      0.77       108
        open       0.74      0.88      0.81       108

    accuracy                           0.79       216
   macro avg       0.80      0.79      0.79       216
weighted avg       0.80      0.79      0.79       216



100%|██████████| 32/32 [00:25<00:00,  1.24it/s]


...


Epoch 8/10
Train Loss: 0.4409
              precision    recall  f1-score   support

      closed       0.92      0.91      0.91       108
        open       0.91      0.92      0.91       108

    accuracy                           0.91       216
   macro avg       0.91      0.91      0.91       216
weighted avg       0.91      0.91      0.91       216



100%|██████████| 32/32 [00:25<00:00,  1.26it/s]



Epoch 9/10
Train Loss: 0.4233
              precision    recall  f1-score   support

      closed       0.96      0.90      0.93       108
        open       0.90      0.96      0.93       108

    accuracy                           0.93       216
   macro avg       0.93      0.93      0.93       216
weighted avg       0.93      0.93      0.93       216



100%|██████████| 32/32 [00:25<00:00,  1.25it/s]



Epoch 10/10
Train Loss: 0.4108
              precision    recall  f1-score   support

      closed       0.96      0.90      0.93       108
        open       0.90      0.96      0.93       108

    accuracy                           0.93       216
   macro avg       0.93      0.93      0.93       216
weighted avg       0.93      0.93      0.93       216



In [18]:
for param in model.features[-2:].parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-5)


In [19]:
for epoch in range(5):
    loss = train_one_epoch(model, train_loader)
    y_true, y_pred, _ = evaluate(model, val_loader)

    print(f"\nFine-Tune Epoch {epoch+1}")
    print("Loss:", round(loss, 4))
    print(classification_report(y_true, y_pred))

100%|██████████| 32/32 [00:27<00:00,  1.16it/s]



Fine-Tune Epoch 1
Loss: 0.3787
              precision    recall  f1-score   support

           0       0.94      0.92      0.93       108
           1       0.92      0.94      0.93       108

    accuracy                           0.93       216
   macro avg       0.93      0.93      0.93       216
weighted avg       0.93      0.93      0.93       216



100%|██████████| 32/32 [00:27<00:00,  1.17it/s]



Fine-Tune Epoch 2
Loss: 0.3533
              precision    recall  f1-score   support

           0       0.95      0.94      0.95       108
           1       0.94      0.95      0.95       108

    accuracy                           0.95       216
   macro avg       0.95      0.95      0.95       216
weighted avg       0.95      0.95      0.95       216



100%|██████████| 32/32 [00:26<00:00,  1.22it/s]



Fine-Tune Epoch 3
Loss: 0.3247
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       108
           1       0.94      0.94      0.94       108

    accuracy                           0.94       216
   macro avg       0.94      0.94      0.94       216
weighted avg       0.94      0.94      0.94       216



100%|██████████| 32/32 [00:25<00:00,  1.25it/s]



Fine-Tune Epoch 4
Loss: 0.297
              precision    recall  f1-score   support

           0       0.95      0.96      0.96       108
           1       0.96      0.95      0.96       108

    accuracy                           0.96       216
   macro avg       0.96      0.96      0.96       216
weighted avg       0.96      0.96      0.96       216



100%|██████████| 32/32 [00:25<00:00,  1.26it/s]



Fine-Tune Epoch 5
Loss: 0.2688
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       108
           1       0.96      0.94      0.95       108

    accuracy                           0.95       216
   macro avg       0.95      0.95      0.95       216
weighted avg       0.95      0.95      0.95       216



In [20]:
y_true, y_pred, y_prob = evaluate(model, test_loader)

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=train_ds.classes))

print("ROC-AUC:", roc_auc_score(y_true, y_prob))


Confusion Matrix:
[[108   2]
 [  5 105]]

Classification Report:
              precision    recall  f1-score   support

      closed       0.96      0.98      0.97       110
        open       0.98      0.95      0.97       110

    accuracy                           0.97       220
   macro avg       0.97      0.97      0.97       220
weighted avg       0.97      0.97      0.97       220

ROC-AUC: 0.9964462809917355


In [21]:
torch.save(model.state_dict(), "eye_state_efficientnet_b0.pth")
print("Eye-state model saved!")


Eye-state model saved!
